In [1]:
# imports
import numpy as np
import matplotlib.pyplot as plt
from numpy.linalg import inv
from scipy.stats import multivariate_normal
import logging
import seaborn as sns
#tips = sns.load_dataset("tips")
sns.set()

from qiskit.circuit.library import TwoLocal
from qiskit.opflow import X, Y, Z, I   
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit import *
from qiskit.algorithms.optimizers import COBYLA
from qiskit_optimization.algorithms import MinimumEigenOptimizer
from qiskit.algorithms import VQE, QAOA
from qiskit.opflow.primitive_ops import PauliSumOp
from qiskit.circuit import Parameter, QuantumCircuit
from qiskit.visualization.array import array_to_latex

In [3]:
# The coreset has been generated elsewhere
coreset = np.array([[ 6.40655667,  0.60980874],
       [-0.47163091,  0.43107638],
       [ 0.52338476, -0.48768696],
       [ 8.3919823 ,  0.19121014],
       [ 1.02084332, -0.2261193 ],
       [ 6.40101943,  2.47664102]])
weights = np.array([216.06327648, 239.04561676, 215.54632493, 143.20444838, 218.40527156, 198.89116006])

In [4]:
# helper functions
def get_mean(coreset):
    dim = len(coreset[0])
    size_coreset = len(coreset)
    mu = np.zeros(dim)
    for i in range(size_coreset):
        mu += coreset[i]
    return mu/size_coreset
    
def get_weighted_mean(coreset, weights):
    dim = len(coreset[0])
    size_coreset = len(coreset)
    mu = np.zeros(dim)
    for i in range(size_coreset):
        mu += coreset[i]*weights[i]
    return mu/sum(weights)

def get_scatter_matrix(coreset):
    dim = len(coreset[0])
    size_coreset = len(coreset)
    T = np.zeros((dim,dim))
    mu = get_mean(coreset)
    for i in range(size_coreset):
        T += np.outer((coreset[i] - mu),(coreset[i] - mu))
    return T
    
def get_weighted_scatter_matrix(coreset, weights):
    dim = len(coreset[0])
    size_coreset = len(coreset)
    T = np.zeros((dim,dim))
    mu = get_weighted_mean(coreset, weights)
    for i in range(size_coreset):
        T += weights[i]*np.outer((coreset[i] - mu),(coreset[i] - mu))
    return T

def get_matrix_inverse(matrix):
    return inv(matrix)

In [5]:
# Helper functions for Hamiltonian creation

def Z_i(i, length):
    """ 
    if index i is in the range 0, ..., length-1, the function returns the operator Z_i
    else: the funtion returns the pauli string consisting of pauli I's only
    length is the number of pauli operators tensorised
    """
    pauli_string = ""
    for j in range(length):
        if i == j:
            pauli_string += "Z"
        else:
            pauli_string += "I"
    return pauli_string

def Z_ij(i, j, length):
    pauli_string = ""
    if i == j:
        pauli_string = Z_i(-1, length) # return 'II...II'
    else:
        for k in range(length):
            if k == i or k == j:
                pauli_string += "Z"
            else:
                pauli_string += "I"
    return pauli_string

In [6]:
# cost for brute-force experiments
def cost_approximate_weighted(coreset, weights, S0, S1):
    w0 = sum(weights[S0])
    w1 = sum(weights[S1])
    W = sum(weights)
    
    w1_div_w0 = 3 - 4*w0/W
    w0_div_w1 = 3 - 4*w1/W

    T_inv = inv(get_weighted_scatter_matrix(coreset, weights))
    cost1 = 0
    for i in S0:
        cost1 += w1_div_w0*weights[i]**2*np.dot(coreset[i], np.dot(T_inv, coreset[i]))
    for i in S1:
        cost1 += w0_div_w1*weights[i]**2*np.dot(coreset[i], np.dot(T_inv, coreset[i]))
    cost2 = 0
    for j in range(len(coreset)):
        for i in range(j):
            if (i in S0 and j in S0):
                cost2 += 2*w1_div_w0*weights[i]*weights[j]*np.dot(coreset[i], np.dot(T_inv, coreset[j]))
            if (i in S1 and j in S1):
                cost2 += 2*w0_div_w1*weights[i]*weights[j]*np.dot(coreset[i], np.dot(T_inv, coreset[j]))
            else:
                cost2 += -2*np.dot(coreset[i], np.dot(T_inv, coreset[j]))

    return -(cost1 + cost2)

In [7]:
all_costs = []
C = coreset
w = weights
for n in range(2**len(C)):
    S0 = [idx for idx,value in enumerate(list(bin(n)[2:].zfill(len(C)))) if value=='0']
    S1 = [idx for idx,value in enumerate(list(bin(n)[2:].zfill(len(C)))) if value=='1']
    all_costs += [cost_approximate_weighted(C, w, S0, S1)]
print(min(all_costs))
index = all_costs.index(min(all_costs))
S0 = [idx for idx,value in enumerate(list(bin(index)[2:].zfill(len(C)))) if value=='0']
S1 = [idx for idx,value in enumerate(list(bin(index)[2:].zfill(len(C)))) if value=='1']
print(S0)
print(S1)
np.sort(all_costs)[0:64]

-1288.7856596657437
[1, 2, 4]
[0, 3, 5]


array([-1288.78565967, -1288.76803101, -1035.99725933, -1035.98725596,
       -1018.6274742 , -1018.62468393, -1007.77352251, -1007.76219852,
        -971.37020283,  -971.3571159 ,  -908.12897265,  -908.12595336,
        -906.41705946,  -906.41685765,  -857.78412356,  -857.76971601,
        -821.18712231,  -821.18669147,  -742.70463632,  -742.69853347,
        -727.29397577,  -727.28810195,  -705.03990338,  -705.03556347,
        -684.90605462,  -684.90194374,  -666.50896189,  -666.50154697,
        -571.53112095,  -571.51040874,  -556.89434936,  -556.87540009,
        -556.7918035 ,  -556.78306796,  -545.25856453,  -545.25505013,
        -536.60463147,  -536.60288001,  -528.15004258,  -528.13500237,
        -504.57933958,  -504.57605421,  -491.37768394,  -491.36718546,
        -491.20221832,  -491.20069589,  -454.2532246 ,  -454.24580113,
        -308.21021375,  -308.2030193 ,  -296.2981831 ,  -296.286364  ,
        -175.31373123,  -175.30889621,    -4.00215599,    -3.99755   ,
      

In [8]:
# Create Hamiltonian for our problem

def create_hamiltonian(coreset, weights):

    paulis = []
    pauli_weights = []
    
    T_inv = inv(get_weighted_scatter_matrix(coreset, weights))

    W = sum(weights)

    for i in range(len(coreset)):
        paulis += [Z_i(-1, len(coreset))]
        pauli_weights += [weights[i]**2*np.dot(coreset[i], np.dot(T_inv, coreset[i]))]
    
        for l in range(len(coreset)):
            paulis += [Z_ij(i,l,len(coreset))]
            pauli_weights += [-2*weights[l]*weights[i]**2*np.dot(coreset[i], np.dot(T_inv, coreset[i]))/W]
            
    for j in range(len(coreset)):
        for i in range(j):
            paulis += [Z_ij(i,j,len(coreset))]
            pauli_weights += [2*weights[i]*weights[j]*np.dot(coreset[i], np.dot(T_inv, coreset[j]))]
            for l in range(len(coreset)):
                paulis += [Z_ij(i,l,len(coreset))]
                pauli_weights += [-2*weights[l]*weights[i]*weights[j]*np.dot(coreset[i], np.dot(T_inv, coreset[j]))/W]
                paulis += [Z_ij(j,l,len(coreset))]
                pauli_weights += [-2*weights[l]*weights[i]*weights[j]*np.dot(coreset[i], np.dot(T_inv, coreset[j]))/W]
            
            
    pauli_op = [([pauli,weight]) for pauli,weight in zip(paulis,pauli_weights)]
    hamiltonian = PauliSumOp.from_list([ op for op in pauli_op])
    # we consider the negative of the hamiltonian since VQE approximates the minimum (and not the maximum)
    hamiltonian = -hamiltonian 
    
    return hamiltonian

In [15]:
## For my experiments I used num_experiments = 30
## For prototyping and testing use a smaller number to avoid long computing times
num_experiments = 5 #for testing
#num_experiments = 30

In [14]:
# general hardware-efficient ansatz
hamiltonian = create_hamiltonian(coreset, weights)

num_qubits = hamiltonian.num_qubits
ansatz = TwoLocal(num_qubits, ['ry','rz'], 'cx', 'linear', reps=1, insert_barriers=True)


qi = QuantumInstance(Aer.get_backend('statevector_simulator'))
    
optimizer = COBYLA(maxiter=100)
vqe = VQE(ansatz, optimizer=optimizer, quantum_instance=qi)

results = []
for i in range(num_experiments):
    results += [vqe.compute_minimum_eigenvalue(hamiltonian)]

In [9]:
# tailored hardware-efficient ansatz
hamiltonian = create_hamiltonian(coreset, weights)

num_qubits = hamiltonian.num_qubits

sub_circ = TwoLocal(num_qubits-1, ['ry','rz'], 'cx', 'linear', reps=1, insert_barriers=True)
sub_inst = sub_circ.to_instruction()

qr = QuantumRegister(num_qubits, 'q')
ansatz = QuantumCircuit(qr)
ansatz.append(sub_inst, [qr[i+1] for i in range(num_qubits -1)])

qi = QuantumInstance(Aer.get_backend('statevector_simulator'))
    
optimizer = COBYLA(maxiter=100)
vqe = VQE(ansatz, optimizer=optimizer, quantum_instance=qi)

results = []
for i in range(num_experiments):
    results += [vqe.compute_minimum_eigenvalue(hamiltonian)]

In [10]:
# TODO: check correctness of usage of QAOA since the results do not approximate the correct solution at all
hamiltonian = create_hamiltonian(coreset, weights)
optimizer = COBYLA(maxiter=100)
qi = QuantumInstance(Aer.get_backend('statevector_simulator'))


from qiskit_optimization.translators import from_ising
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit.opflow.primitive_ops import PauliSumOp

qp   = from_ising(hamiltonian)
conv = QuadraticProgramToQubo()
qubo = conv.convert(qp)

results = []
for i in range(num_experiments):
    optimizer = COBYLA(maxiter=100)
    qi = QuantumInstance(Aer.get_backend('statevector_simulator'))
    # reps=depth
    qaoa = QAOA(optimizer=optimizer, reps=6, quantum_instance=qi)

    algorithm = MinimumEigenOptimizer(qaoa)

    result = algorithm.solve(qubo)
    results += [result]